In [6]:
import pandas as pd

pd.set_option("display.precision", 2)

In [7]:
import glob

PATH = glob.glob("../executions/MNQM2022/*.csv")[0]
PATH

'../executions/MNQM2022\\20220318-20220427.csv'

In [8]:
def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    df_grouped = df.groupby("date").sum()[["realized_pnl", "px_side"]]

    df_grouped["$_per_px"] = df_grouped["realized_pnl"] / df_grouped["px_side"]
    df_grouped["realized_pnl_cum"] = df_grouped["realized_pnl"].cumsum()
    df_grouped["profit"] = df[df["realized_pnl"] > 0].groupby("date").count()["realized_pnl"]
    df_grouped["loss"] = df[df["realized_pnl"] < 0].groupby("date").count()["realized_pnl"]
    df_grouped.loc["Total"] = df_grouped.sum(numeric_only=True)

    return df_grouped


df_exec = get_executions_df(PATH)
df_exec

,realized_pnl,px_side,$_per_px,realized_pnl_cum,profit,loss
date,,,,,,
2022-03-18,2326.02,320.21,7.26,2326.02,14.0,5.0
2022-03-21,-1456.60,-306.44,4.75,869.42,12.0,15.0
2022-03-22,943.20,117.32,8.04,1812.62,3.0,2.0
2022-03-23,-261.50,-47.26,5.53,1551.12,5.0,9.0
2022-03-24,717.28,145.17,4.94,2268.40,6.0,3.0
2022-03-25,-69.18,-76.45,0.90,2199.22,17.0,12.0
2022-03-28,649.84,53.15,12.23,2849.06,11.0,8.0
2022-03-29,419.24,47.71,8.79,3268.30,13.0,5.0
2022-03-30,-299.50,30.12,-9.94,2968.80,21.0,13.0


In [9]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    df_pos = df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]]
    df_neg = df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]

    return df.groupby("date").sum()[cols] \
        .join(df_pos, rsuffix="_pos") \
        .join(df_neg, rsuffix="_neg") \
        .join(df.groupby("date").count()[["realized_pnl"]], rsuffix="_count")


get_executions_ungroup(PATH)

,quantity,realized_pnl,px_side,realized_pnl_pos,realized_pnl_neg,realized_pnl_count
date,,,,,,
2022-03-18,57,2326.02,320.21,2943.51,-617.49,19
2022-03-21,90,-1456.60,-306.44,3122.90,-4579.50,27
2022-03-22,20,943.20,117.32,1809.46,-866.26,5
2022-03-23,50,-261.50,-47.26,1888.12,-2149.62,14
2022-03-24,23,717.28,145.17,1050.76,-333.48,9
2022-03-25,87,-69.18,-76.45,2184.02,-2253.20,29
2022-03-28,69,649.84,53.15,2311.20,-1661.36,19
2022-03-29,59,419.24,47.71,1323.34,-904.10,18
2022-03-30,150,-299.50,30.12,2425.98,-2725.48,34


In [10]:
import plotly.express as px

fig = px.line(df_exec, x=df_exec.index, y="realized_pnl_cum")
fig.show()
